In [111]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

In [112]:
list_departement=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Liste-Excel-des-departements-francais.xlsx")


In [113]:
path=r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - Reseaux de distibution\GSA_GSS 2020"
gsa=pd.read_csv(path+r"\GSA_2022_clean.csv")
gsa = gpd.GeoDataFrame(
    gsa, geometry=gpd.points_from_xy(gsa.X__WGS84_, gsa.Y__WGS84_)).set_crs("EPSG:4326").to_crs("EPSG:2154")
gss=pd.read_pickle(path+r"\GSA_GSS_actif_iris_2020_geo_gss.pkl")
gss = gpd.GeoDataFrame(
    gss, geometry=gpd.points_from_xy(gss.X__WGS84_, gss.Y__WGS84_)).set_crs("EPSG:4326").to_crs("EPSG:2154")

In [114]:
logs=pickle.load(open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\\logs.pkl","rb"))

In [115]:
for i ,row in list_departement.iterrows():
    if row["Département"].lower()=="rhône":
        try:
            try:
                dep=int(row['N°'])
                gsa_dep=gsa[gsa['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]
                gss_dep=gss[gss['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]
            except:
                gsa_dep=gsa.copy()
                gss_dep=gss.copy()
            place_name=row["Département"].lower().strip()
            path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+r"\\"+row["Département"].lower()

            try:gsa_ass=pickle.load(open(path+r"\\association\\GSA_association.pkl","rb"))
            except:pass

            try:gss_ass=pickle.load(open(path+r"\\association\\GSS_association.pkl","rb"))
            except:pass

            gsa_dep=gsa[gsa['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]
            gss_dep=gss[gss['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]

            if len(gsa_dep)>0:
                shops=pickle.load(open(path+r"\\raw data\\shops.pkl",'rb')).to_crs('EPSG:2154')
                gsa_comp=gsa_dep.iloc[[i for i in range(len(gsa_dep)) if i not in gsa_ass['gsa'].values]][['Type_Mag' ,  'Nom_Enseigne' ,'CP' ,"N__Nom_de_voie","geometry"]].to_crs('EPSG:4326')
                pickle.dump(gsa_comp,open(path+r"\\processed data\\gsa_comp.pkl","wb"))
                logs[place_name].update({"gsa_comp":len(gsa_comp)})
            if  len(gss_dep)>0:
                shops=pickle.load(open(path+r"\\raw data\\shops.pkl",'rb')).to_crs('EPSG:2154')
                gss_comp=gss_dep.iloc[[i for i in range(len(gss_dep)) if i not in gss_ass['gss'].values]][['Type_Mag' ,  'Nom_Enseigne' ,'CP' ,"N__Nom_de_voie","geometry"]].to_crs('EPSG:4326')
                pickle.dump(gss_comp,open(path+r"\\processed data\\gss_comp.pkl","wb"))
                logs[place_name].update({"gss_comp":len(gss_comp)})
        except:
            print(place_name)
